In [17]:
from collections import Counter
from marked_words_occ import marked_words as get_marked_words_occ
from marked_words_prob_kevin import marked_words as get_marked_words_no_occ
# from marked_words_prob_jenn import marked_words as get_marked_words_no_occ
from nltk.tokenize import sent_tokenize, word_tokenize

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [18]:
df_10 = pd.read_csv('original_marked_personas_generated_personas_gpt-3.5-turbo-0125_10.csv')

In [19]:
df_100 = pd.read_csv('original_marked_personas_generated_personas_gpt-3.5-turbo-0125_100_06-24-2024, 10:30:39.csv')

In [20]:
# df_100_for_loop = pd.read_csv('original_marked_personas_generated_personas_gpt-3.5-turbo-0125_100_06-29-2024, 13:28:28.csv')
# df_10_for_loop_1 = pd.read_csv('original_marked_personas_generated_personas_gpt-3.5-turbo-0125_10_06-26-2024, 16:30:49.csv') 

# df_10_for_loop = pd.read_csv('original_marked_personas_generated_personas_gpt-3.5-turbo-0125_10_06-26-2024, 17:13:53.csv')

df_200_for_loop = pd.read_csv('original_marked_personas_generated_personas_gpt-3.5-turbo-0125_200_06-30-2024, 12:47:15.csv')

In [21]:
def pprint(dic):
    full_list = []
    for word in sorted(dic,key=lambda x: x[1],reverse=True):
        if word[0].lower() == 'latina':
            print('LATINA')
        full_list.append(word[0])
    return full_list

In [22]:
def compute_marked_words(df, prior=True, frac_words=1):
    dv3_mw = {}
    for race in df['race'].unique():
    #     print('\n Top words for %s \n-------' % race)
        outs = pprint(get_marked_words_no_occ(df, [race], ['race'],['a White'], prior=prior, frac_words=frac_words))
        dv3_mw[race] = outs
    temps = []
    for race in df['race'].unique():
    #     print('\n Top words for %s \n-------' % race)
        temp = pprint(get_marked_words_no_occ(df, ['a White'], ['race'],[race], prior=prior, frac_words=frac_words))
        temps.extend(temp)
    seen = Counter(temps).most_common()
    dv3_mw['a White']=[w for w, c in seen if c == 4]


    for race in df['gender'].unique():
    #     print('\n Top words for %s \n-------' % race)
        outs = pprint(get_marked_words_no_occ(df, [race], ['gender'],['M'], prior=prior, frac_words=frac_words))
        dv3_mw[race] = outs
    temps = []
    for race in df['gender'].unique():
    #     print('\n Top words for %s \n-------' % race)
        temp = pprint(get_marked_words_no_occ(df, ['M'], ['gender'],[race], prior=prior, frac_words=frac_words))
        temps.extend(temp)

    seen = Counter(temps).most_common()
    dv3_mw['M']=[w for w, c in seen if c == 2]
        
        
    # Top words for intersectional groups
    for race in df['race'].unique():
        for gen in ['M', 'N','F']:
            dv3_mw[race+gen] = pprint(get_marked_words_no_occ(df, [race, gen], ['race', 'gender'],['a White','M'], prior=prior, frac_words=frac_words))

    marked_word_similarity = dict()
    temp = list()
    for key in list(dv3_mw.keys()):
        marked_word_similarity[key] = list()
        for word in dv3_mw[key]:
            temp.append(word_tokenize(word))
    if len(temp) > 0:
        data = temp
    # print(data)
    return dv3_mw


In [23]:
def compare_marked_words_across_gens(frac_words, prior=True, test_words_enclosed=False):
    dv3_mw_10 = compute_marked_words(df_10, prior=prior, frac_words=frac_words)
    dv3_mw_100  = compute_marked_words(df_100, prior=prior, frac_words=frac_words)
    # dv3_mw_100_for = compute_marked_words(df_100_for_loop, prior=prior, frac_words=frac_words)
    # dv3_mw_10_for = compute_marked_words(df_10_for_loop, prior=prior, frac_words=frac_words)
    dv3_mw_200_for = compute_marked_words(df_200_for_loop, prior=prior, frac_words=frac_words)

    for key in list(dv3_mw_10.keys()):
        if key == 'b':
            continue
        if len(dv3_mw_100[key]) == 0:
            continue
        print(key)
        print(len(dv3_mw_10[key]), '| 10 |', sorted(dv3_mw_10[key]))
        print(len(dv3_mw_100[key]), '| 100 |', sorted(dv3_mw_100[key]))
        # try:
        # print(len(dv3_mw_100_for[key]), '| 100 for |', sorted(dv3_mw_100_for[key]))
        print(len(dv3_mw_200_for[key]), '| 200 for |', sorted(dv3_mw_200_for[key]))
        # print(len(dv3_mw_10_for[key]), '| 10 for |', sorted(dv3_mw_10_for[key]))
    
    if test_words_enclosed:
        passed = True
        for key in list(dv3_mw_10.keys()):
            if key == 'b':
                continue
            if len(dv3_mw_100[key]) == 0:
                continue
            for word in dv3_mw_10[key]:
                if word not in dv3_mw_100[key]:
                    print(f"FAILED on {word} in {key}")
                    passed = False
        if passed:
            print("PASSED")
    return dv3_mw_10, dv3_mw_100, dv3_mw_200_for#, dv3_mw_10_for, dv3_mw_100_for
    # return dv3_mw_10, dv3_mw_100, dv3_mw_100_for, dv3_mw_10_for, dv3_mw_100_for

In [24]:
# default
default_x = compare_marked_words_across_gens(1, True, True)

c: 0.005078125, p: 35845.5, g1: 6337.0, g2: 6337.0, r1: 0.02872462201159855, r2: 0.02872462201159855
c: 0.005078125, p: 33864.0, g1: 7208.0, g2: 5919.0, r1: 0.023857606132075476, r2: 0.029053155093765843
c: 0.005078125, p: 35232.0, g1: 6627.0, g2: 6197.0, r1: 0.026997510185604347, r2: 0.02887082459254478
c: 0.005078125, p: 33864.0, g1: 7232.0, g2: 5919.0, r1: 0.02377843266039823, r2: 0.029053155093765843
c: 0.005078125, p: 34343.5, g1: 2871.0, g2: 6026.0, r1: 0.06074558897161268, r2: 0.02894135179845669
c: 0.005078125, p: 33637.0, g1: 4585.0, g2: 5902.0, r1: 0.03725471987459106, r2: 0.028941526707048457
c: 0.005078125, p: 35845.5, g1: 6337.0, g2: 6337.0, r1: 0.02872462201159855, r2: 0.02872462201159855
c: 0.005078125, p: 33864.0, g1: 5919.0, g2: 7208.0, r1: 0.029053155093765843, r2: 0.023857606132075476
c: 0.005078125, p: 35232.0, g1: 6197.0, g2: 6627.0, r1: 0.02887082459254478, r2: 0.026997510185604347
c: 0.005078125, p: 33864.0, g1: 5919.0, g2: 7232.0, r1: 0.029053155093765843, r2: 0